In [8]:
import findspark

findspark.init()

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, sum

# Create a Spark session
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

# Task 1: Load data from HDFS
results = spark.read.csv("hdfs:///outpucik/results.csv", header=True, inferSchema=True)
races = spark.read.csv("hdfs:///outpucik/races.csv", header=True, inferSchema=True)
drivers = spark.read.csv("hdfs:///outpucik/drivers.csv", header=True, inferSchema=True)

In [10]:
# Task 2: Check columns and data types, count records
print("Results schema:")
results.printSchema()
results_count = results.count()
print(f"Number of records in results: {results_count}")

print("\nRaces schema:")
races.printSchema()
races_count = races.count()
print(f"Number of records in races: {races_count}")

print("\nDrivers schema:")
drivers.printSchema()
drivers_count = drivers.count()
print(f"Number of records in drivers: {drivers_count}")

Results schema:
root
 |-- resultId: integer (nullable = true)
 |-- raceId: integer (nullable = true)
 |-- driverId: integer (nullable = true)
 |-- constructorId: integer (nullable = true)
 |-- number: integer (nullable = true)
 |-- grid: integer (nullable = true)
 |-- position: integer (nullable = true)
 |-- positionText: string (nullable = true)
 |-- positionOrder: integer (nullable = true)
 |-- points: double (nullable = true)
 |-- laps: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- milliseconds: integer (nullable = true)
 |-- fastestLap: integer (nullable = true)
 |-- rank: integer (nullable = true)
 |-- fastestLapTime: string (nullable = true)
 |-- fastestLapSpeed: string (nullable = true)
 |-- statusId: integer (nullable = true)

Number of records in results: 23777

Races schema:
root
 |-- raceId: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- round: integer (nullable = true)
 |-- circuitId: integer (nullable = true)
 |-- name: string (nul

In [11]:
# Task 3: Merge data and count rows

races = races.withColumnRenamed("time", "race_time")
drivers = drivers.withColumnRenamed("number", "driver_number") \
                 .withColumnRenamed("url", "driver_url")
results = results.withColumnRenamed("time", "results_time") \
                 .withColumnRenamed("number", "results_number")

merged_data = results.join(races, "raceId").join(drivers, "driverId")
merged_count = merged_data.count()
print(f"\nNumber of rows after merging: {merged_count}")

# Task 4: Convert to Parquet and save to HDFS
merged_data.write.parquet("hdfs:///outpucik/merged_data.parquet")


Number of rows after merging: 23777


In [12]:
# Task 5: Calculate average speed for fastest laps in 2008 and 2009
average_speed = (
    results
    .join(races, "raceId")
    .filter((col("year") == 2008) | (col("year") == 2009))
    .groupBy("driverId")
    .agg(avg("fastestLapSpeed").alias("averageSpeed"))
)
average_speed.sort('averageSpeed', ascending=False).show()

+--------+------------------+
|driverId|      averageSpeed|
+--------+------------------+
|      69|           211.786|
|      24|          206.9228|
|       8|204.43448571428573|
|       4|           203.494|
|      11|         203.46025|
|       2|203.35114285714278|
|       9|203.32755882352942|
|     154|203.24066666666667|
|      20| 203.1014838709678|
|     155|          203.0635|
|       1| 203.0331764705882|
|       3| 203.0095428571429|
|      13|202.94729629629632|
|       5|202.80760606060605|
|      22| 202.3576857142857|
|      10|202.17048387096776|
|      17| 201.9461212121212|
|      67|           201.713|
|      18|201.68184848484853|
|       6|201.25382352941173|
+--------+------------------+
only showing top 20 rows



In [13]:
merged_data.filter((col('nationality') == 'American') & (col("year") == 1960)).select('points').show()

+------+
|points|
+------+
|   0.0|
|   0.0|
|   0.0|
|   4.0|
|   1.0|
|   0.0|
|   0.0|
|   0.0|
|   0.0|
|   8.0|
|   6.0|
|   4.0|
|   3.0|
|   2.0|
|   1.0|
|   0.0|
|   0.0|
|   0.0|
|   0.0|
|   0.0|
+------+
only showing top 20 rows



In [ ]:
# Task 6: Group data by nationality, year, race name, and calculate total points and max lap speed
grouped_data = (
    merged_data
    .groupBy("nationality", "year", "name")
    .agg(
        sum("points").alias("totalPoints"),
        max("fastestLapSpeed").alias("maxLapSpeed")
    )
    .na.drop(subset=["maxLapSpeed"])
)
grouped_data.sort('totalPoints', 'maxLapSpeed', ascending=False).show()

In [ ]:
# Task 7: Create a Spark SQL view
grouped_data.createOrReplaceTempView("race_stats")

In [ ]:
# Task 8: Count rows and max points per nationality using Spark SQL view
sql_query = "SELECT nationality, COUNT(*) as rowCount, MAX(totalPoints) as maxPoints FROM race_stats GROUP BY nationality"
result_view = spark.sql(sql_query)
result_view.show()

In [ ]:
result_view.filter(col('nationality') == 'German').show()